In [1]:
!git clone https://github.com/Hallucination-LLM/Research.git

fatal: destination path 'Research' already exists and is not an empty directory.


In [ ]:
!cd Research/api && pip install -r requirements.txt

In [96]:
import subprocess

log_file = open("uvicorn_log.txt", "w")

command = ["uvicorn", "main:app", "--host", "127.0.0.1", "--port", "8000"]
process = subprocess.Popen(command, cwd="Research/api", stdout=log_file, stderr=log_file)

In [113]:
# Read and display the logs
with open("uvicorn_log.txt", "r") as log_file:
    logs = log_file.read()
    print(logs)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2024-08-20 16:27:40.966690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 16:27:40.966761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 16:27:40.968330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO:     Started server process [821]
INFO:     Waiting for application startup.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)



# Need to check function above if log **INFO:     Application startup complete.**  apear app works


In [84]:
#process.terminate()  # This will stop the process

In [114]:
import requests 


URL = "http://localhost:8000"

endpoint = "/test"

response = requests.get(f'{URL}{endpoint}', timeout=20)

response.json()

{'message': 'Test endpoint is working'}

In [115]:
endpoint = "/generate"

body = {
    "input": "What is the capital of France?",
    "use_dola": True,
}

response = requests.post(f'{URL}{endpoint}', json=body, timeout=20)
response.json()

{'message': 'Generated content: The capital of France is **Paris**.<end_of_turn>\n<eos>'}

In [116]:
endpoint = "/detect_hallu"

body = {
    "user_input": "What is the capital of France?",
    "system_msg": """
        You are an assistant in the systems monitoring team. Your job will be to answer
        questions accurately based on the given context. IF YOU DO NOT KNOW THE ANSWER
        TO THE QUESTION, RETURN THE ANSWER THAT THE CONTEXT DOES NOT INCLUDE THE GIVEN INFORMATION.
        Don't make things up.
        Answer only on the basis of the knowledge contained in the context. If the context is incomplete, don't lead conjecture.
        Answers should be clear and precise. Pay special attention to the names of applications, services, tools, components
        - it is crucial to return information that is consistent for the subject. Context will be given in portions (chunks), your
        task is to infer which information is most closely related to the question, even if any portion of the context,
        seems to answer the question, always check if this is exactly what the user might have meant. If necessary,
        then include context from multiple chunks. Each chunk contains basic information at the beginning.
        """,
}

response = requests.post(f'{URL}{endpoint}', json=body, timeout=20)
response.json()

{'input': 'What is the capital of France?', 'hallucination': 1}

In [134]:
process.terminate()

In [136]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

0

# Second Method


In [137]:
%cd Research/api

/kaggle/working/Research/api


In [138]:
from app.services.ai_processing_service import AIProcessingService
from app.fastapi_models import RequestModel, DetectHalluRequest

In [139]:
ai_service = AIProcessingService()
await ai_service.on_startup()

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [140]:
request = RequestModel(input="What is the capital of France?", use_dola=True)

response = await ai_service.generate(request)

In [141]:
response

ResponseModel(message='Generated content: The capital of France is **Paris**.<end_of_turn>\n<eos>')

In [142]:
request = DetectHalluRequest(user_input="What is the capital of France?")

response = await ai_service.detect_hallu(request)

In [143]:
response

DetectHalluResponse(input='What is the capital of France?', hallucination=1)

In [144]:
del ai_service

In [147]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

0